In [1]:
import os
os.chdir('../')
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from evaluator.gpt_evaluator import FinancialDataProcessor
import time
import ast
import yfinance as yf

/home/ubuntu/miniforge3/envs/multimodal/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [1]:
def wait_for_completion(job_id, processor, poll_interval=50):
    status = processor.check_status(job_id)
    while status.status not in ["completed", "failed"]:
        print(f"Current status: {status}. Waiting for {poll_interval} seconds...")
        time.sleep(poll_interval)
        status = processor.check_status(job_id)
    return status.status

# Process the financial Data

In [30]:
def process_data(file_name, input_dir, output_dir):
    # Load the data
    df = pd.read_csv(os.path.join(input_dir, file_name))

    # Step 1: Filter out rows where 'text' starts with "Access to this page has been denied"
    df_filtered = df[~df['text'].str.startswith('Access to this page has been denied', na=False)].copy()

    # Step 2: Ensure the timestamp column is in datetime format
    df_filtered['timestamp'] = pd.to_datetime(df_filtered['timestamp'])

    # Step 3: Convert the timestamp to just the date part
    df_filtered['date'] = df_filtered['timestamp'].dt.date

    # Step 4: Define URLs to match MarketWatch, Reuters, and Investors in order of priority
    url_priority = ['marketwatch', 'reuters', 'investors', 'benzinga', 'zacks', 'fool', 'thestreet', 'forbes']

    # Step 5: Assign priority based on the URL. Check if the URL contains any of the specified keywords.
    df_filtered['priority'] = df_filtered['url'].apply(lambda x: next((i for i, kw in enumerate(url_priority) if kw in x.lower()), len(url_priority)))

    # Step 6: Sort by date and priority
    df_filtered = df_filtered.sort_values(by=['date', 'priority'])

    # Step 7: Group by date and choose up to 5 URLs (priority first, no duplicates per day)
    def sample_top_five_per_day(group):
        seen_sites = set()
        sampled = pd.DataFrame()  # Initialize an empty DataFrame for sampled rows
        
        for idx, row in group.iterrows():
            site = next((kw for kw in url_priority if kw in row['url'].lower()), None)
            if site and site not in seen_sites:
                sampled = pd.concat([sampled, group.loc[[idx]]])  # Append the row as DataFrame
                seen_sites.add(site)
            if len(sampled) >= 5:  # Stop once 5 distinct sites are found
                break
        
        # If less than 5, fill with non-priority but ensure no duplicate sources
        remaining_needed = 5 - len(sampled)
        if remaining_needed > 0:
            non_priority_urls = group[~group['url'].apply(lambda x: any(kw in x.lower() for kw in seen_sites))]
            sampled_non_priority = non_priority_urls.head(remaining_needed)
            sampled = pd.concat([sampled, sampled_non_priority])  # Append non-priority URLs

        return sampled

    sampled_data = df_filtered.groupby('date', group_keys=False).apply(sample_top_five_per_day)

    # Step 8: Sort the data by date
    sampled_data = sampled_data.sort_values(by='date')

    # Step 9: Save the resulting DataFrame to a new file
    output_file = os.path.join(output_dir, file_name)
    sampled_data.to_csv(output_file, index=False)



# Financial Data

In [31]:
def get_gpt_results(output_dir, results_dir, file_name):
    financial_data_processor = FinancialDataProcessor()

    data = pd.read_csv(os.path.join(output_dir, file_name))

    jsonl_path = os.path.join(results_dir, "batch.jsonl")
    output_path = os.path.join(results_dir, "data.txt")

    batch_object_id = financial_data_processor.create_and_run_batch_job(data, jsonl_path, ticker_column="ticker", date_column="timestamp", input_text_column="text")

    job_status = wait_for_completion(batch_object_id, financial_data_processor)

    print("Job status:", job_status)
    if job_status == "completed":
        print("Batch job completed successfully!")
        gpt_outputs = financial_data_processor.check_status_and_parse(batch_object_id , output_path)
        dict_list = [ast.literal_eval(item) for item in gpt_outputs]
        df = pd.DataFrame(dict_list)
        # df['text'] = df.apply(lambda x: {"ticker": x['ticker'], "summary": x['summary']}, axis=1)
        # df = df.drop(columns=['summary', 'keywords'])
        df.to_csv(os.path.join(results_dir, file_name), index=False)
    


In [2]:
def combined_result(result_dir, combined_dir, file_name):
    # Read the CSV file
    df = pd.read_csv(os.path.join(result_dir, file_name))
    
    # Drop rows where 'summary' or 'keywords' are NaN
    df_filtered = df.dropna(subset=['summary', 'keywords'])
    
    # Function to combine summaries and keywords
    def combine_random(group):
        # Randomly sample 3 or fewer summaries and keywords (if fewer than 3, choose what is available)
        chosen_rows = group.sample(n=min(3, len(group)), random_state=1)
        
        # Combine the selected summaries
        combined_summary = ' '.join(chosen_rows['summary'])
        
        # Combine keywords, split by commas, remove duplicates using a set, and then join them back
        combined_keywords = ', '.join(sorted(set(', '.join(chosen_rows['keywords']).split(', '))))
        
        # Return a dictionary containing combined summary and keywords
        return pd.Series({'text': combined_summary, 'keywords': combined_keywords})

    # Group by 'ticker' and 'date', then apply the combine function
    df_grouped = df_filtered.groupby(['ticker', 'date']).apply(combine_random).reset_index()

    # Convert 'date' column in df_grouped to datetime
    df_grouped['date'] = pd.to_datetime(df_grouped['date'])

    # Extract the ticker from the file name, and find the date range for stock data
    ticker = file_name.split(".")[0]
    start_date = df_grouped['date'].min()
    end_date = df_grouped['date'].max()
    print("ticker: ", ticker, "start_date: ", start_date, "end_date: ", end_date)
    
    # Fetch the stock data for the given date range
    df_stock = yf.download(ticker, start=start_date, end=end_date)

    # Reset the index to make 'Date' a column and work with a copy to avoid SettingWithCopyWarning
    df_stock = df_stock.reset_index()[['Date', 'Close']].copy()

    # Rename 'Date' to 'date' to match with the existing DataFrame
    df_stock.rename(columns={'Date': 'date'}, inplace=True)

    # Merge the two DataFrames on 'date'
    df_merged = pd.merge(df_grouped, df_stock, on='date', how='left')

    df_merged = df_merged.dropna(subset=['Close'])

    # Display the merged DataFrame
    print("Merged DataFrame with 'close' column added (rows with NaN 'close' dropped):")
    print(df_merged.head())

    # Save the merged DataFrame to a CSV file
    df_merged.to_csv(os.path.join(combined_dir, file_name), index=False)


In [3]:
input_dir = "/home/ubuntu/multimodal/Data/financial-raw/"
output_dir = "/home/ubuntu/multimodal/Data/financial-processed/"
results_dir = "/home/ubuntu/multimodal/Data/financial-gpt/"
combined_dir = "/home/ubuntu/multimodal/Data/financial-grouped"

file_names = ["AAPL.csv", "AMZN.csv", "AMD.csv", "NFLX.csv", "AVGO.csv"]
# file_names = ["AVGO.csv"]
# file_names = ["CRM.csv"]
for file_name in file_names:
    # process_data(file_name, input_dir, output_dir)
    # get_gpt_results(output_dir, results_dir, file_name)
    combined_result(results_dir, combined_dir, file_name)

/tmp/ipykernel_1518700/1724619003.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_grouped = df_filtered.groupby(['ticker', 'date']).apply(combine_random).reset_index()


ticker:  AAPL start_date:  2022-03-01 00:00:00 end_date:  2024-04-29 00:00:00


[*********************100%***********************]  1 of 1 completed


Merged DataFrame with 'close' column added (rows with NaN 'close' dropped):
  ticker       date                                               text  \
0   AAPL 2022-03-01  Apple Inc. has halted product sales in Russia ...   
1   AAPL 2022-03-02  Apple halted sales of its products in Russia f...   
2   AAPL 2022-03-03  Apple stock climbed 2.1% to 166.56, moving tow...   
3   AAPL 2022-03-04  In February, the U.S. stock market experienced...   
6   AAPL 2022-03-07  Apple co-founder Steve Wozniak discussed Bitco...   

                                            keywords       Close  
0  Apple, Russia, S&P 500, Ukraine crisis, halt s...  163.199997  
1  AAPL, Apple, Apple Pay, Russia, Ukraine, Ukrai...  166.559998  
2  AAPL, Apple, Russia, Ukraine, Winslow Capital ...  166.229996  
3  AAPL, Apple, Dow Jones, Russia-Ukraine, Russia...  163.169998  
6  Apple, Beats, Bitcoin, HBCU, Living In Legacy,...  159.300003  


/tmp/ipykernel_1518700/1724619003.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_grouped = df_filtered.groupby(['ticker', 'date']).apply(combine_random).reset_index()
[*********************100%***********************]  1 of 1 completed


...

ticker:  AMD start_date:  2022-03-04 00:00:00 end_date:  2024-04-28 00:00:00
Merged DataFrame with 'close' column added (rows with NaN 'close' dropped):
  ticker       date                                               text  \
0    AMD 2022-03-04  AMD is set to release its first-quarter earnin...   
1    AMD 2022-03-07  Advanced Micro Devices (AMD) expects to releas...   
2    AMD 2022-03-08  Advanced Micro Devices (AMD) has gained 31.1% ...   
3    AMD 2022-03-09  Advanced Micro Devices (AMD) announced an addi...   
4    AMD 2022-03-10  Shares of Advanced Micro Devices, Inc. (AMD) a...   

                                            keywords       Close  
0  AMD, CPU market share, Intel, console sales, e...  108.410004  
1  AMD, ASML, Applied Materials, EPYC processors,...  102.949997  
2  AMD, GPU, Infinity Cache, Mac Pro, RDNA 2, Rad...  105.529999  
3  AMD, Advanced Micro Devices, Apple, Ryzen Thre...  111.050003  
4  AMD, US Treasury yield, earnings, estimate, fi...  106.459999  


/tmp/ipykernel_1518700/1724619003.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_grouped = df_filtered.groupby(['ticker', 'date']).apply(combine_random).reset_index()
[*********************100%***********************]  1 of 1 completed


ticker:  NFLX start_date:  2022-03-04 00:00:00 end_date:  2024-04-29 00:00:00
Merged DataFrame with 'close' column added (rows with NaN 'close' dropped):
  ticker       date                                               text  \
0   NFLX 2022-03-04  In April 2022, Netflix Inc. (NFLX) experienced...   
3   NFLX 2022-03-07  Netflix has announced that it is suspending it...   
4   NFLX 2022-03-08  Streaming leader Netflix Inc (NFLX) has had su...   
5   NFLX 2022-03-09  Netflix (NFLX) gained 5% following an upgrade ...   
6   NFLX 2022-03-10  The PGA Tour announced a Netflix docuseries wi...   

                                            keywords       Close  
0  April 2022, NFLX, Netflix, S&P 500, buy, decli...  361.730011  
3  Netflix, Russia, Ukraine, conflict, invasion, ...  350.260010  
4  Assassin’s Creed, League of Legends, Netflix, ...  341.760010  
5  NFLX, Netflix, Wedbush, bearish, bullish, earn...  358.790009  
6  Box to Box Films, Formula 1: Drive to Survive,...  356.769989  

/tmp/ipykernel_1518700/1724619003.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_grouped = df_filtered.groupby(['ticker', 'date']).apply(combine_random).reset_index()
[*********************100%***********************]  1 of 1 completed

ticker:  AVGO start_date:  2022-03-03 00:00:00 end_date:  2024-04-28 00:00:00
Merged DataFrame with 'close' column added (rows with NaN 'close' dropped):
  ticker       date                                               text  \
0   AVGO 2022-03-03  A whale has taken a bearish stance on Broadcom...   
1   AVGO 2022-03-04  Broadcom Inc (AVGO) is a semiconductor giant s...   
2   AVGO 2022-03-07  Shares of Broadcom Inc. (AVGO) gained 3% after...   
3   AVGO 2022-03-08  Broadcom Inc. (AVGO) is a semiconductor and in...   
4   AVGO 2022-03-09  Over the past three months, shares of Broadcom...   

                                            keywords      Close  
0  AVGO, Broadcom, Truist Securities, bearish sta...  57.860001  
1  5G, AVGO, Broadcom, Nasdaq bear market, PCIe G...  59.598999  
2  AVGO, Broadcom, Q1 fiscal 2022, Zacks Consensu...  57.070000  
3  Broadcom, Infrastructure software, Semiconduct...  57.595001  
4  AVGO, Broadcom, Electronics Semiconductors, VC...  59.702000  


In [6]:
import pandas as pd
import glob

# Step 1: Retrieve all CSV file paths in the specified directory
file_paths = glob.glob('/home/ubuntu/multimodal/Data/financial-gpt/*.csv')

# Step 2: Read all CSV files and concatenate them into a single DataFrame
all_data = pd.concat((pd.read_csv(file) for file in file_paths))

# Step 3: Ensure the 'date' column is in datetime format for proper grouping
all_data['date'] = pd.to_datetime(all_data['date'])

# Step 4: Replace NaN in the 'text' column with blank strings
all_data['text'] = all_data['text'].fillna('')

# Step 5: Group by the 'date' column and concatenate the 'text' column for each group
grouped_data = all_data.groupby('date').agg({'text': '\n'.join})

# Step 6: Reset the index to flatten the DataFrame
grouped_data = grouped_data.reset_index()

# Step 7: Save the aggregated data to a CSV file
grouped_data.to_csv('results_grouped_by_date.csv', index=False)

# Print the shape of the resulting DataFrame
print(grouped_data.shape)
print("Data grouped by date and saved to 'results_grouped_by_date.csv'")


(791, 2)
Data grouped by date and saved to 'results_grouped_by_date.csv'


In [2]:

import pandas as pd

# Define the stock ticker and the date range
ticker = 'AAPL'
start_date = '2022-03-22'
end_date = '2024-03-26'

# Fetch the historical data
aapl_data = yf.download(ticker, start=start_date, end=end_date)

# Extract the 'Close' column
aapl_close = aapl_data['Close']

# Display the Close values
print(aapl_close)


[*********************100%***********************]  1 of 1 completed

Date
2022-03-22    168.820007
2022-03-23    170.210007
2022-03-24    174.070007
2022-03-25    174.720001
2022-03-28    175.600006
                 ...    
2024-03-19    176.080002
2024-03-20    178.669998
2024-03-21    171.369995
2024-03-22    172.279999
2024-03-25    170.850006
Name: Close, Length: 505, dtype: float64
